In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns

# 맵 
import json
import geopandas as gpd
from shapely.geometry import Point,LineString,Polygon

# matplotlib
from adjustText import adjust_text
import matplotlib.patches as mpatches
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
from datetime import datetime

# 진행상황
from IPython.display import display
from tqdm import tqdm

# 폰트
import matplotlib.font_manager as fm
import matplotlib as mpl

mpl.rcParams['axes.unicode_minus']=False
path = 'malgun.ttf'
fontprop = fm.FontProperties(fname=path, size=13)

# 경고제어
import warnings
warnings.filterwarnings('ignore')

In [2]:
# indcd_{분류}_yn 가 Y 인것중에
# age 별로 사용자 비중을 비교하고
# sales 별로 매출액도 비교하고
# loan 해서 대출액이 많은 지도 보고
# rpr_per_age 로 대표자의 연령대도 보고
# rpr_per_gender로 대표자의 성별도 보고
# smbiz_yn_cnt  소상공인 사업장인지 보고
# runout_cnt 휴폐점인지도 보고
# found 몇년 미만의 사업장인지도 보고
# ws_cnt로 격자내 사업장이 많은지도 보고 

In [3]:
store=pd.read_csv('17.용인시_소상공인_매출정보.csv')
geo_store=gpd.read_file('16.용인시_소상공인_매출정보.geojson')
dong=gpd.read_file('12.용인시_법정경계(읍면동).geojson')

geo_store=geo_store.dropna(axis=1)

In [78]:
for idx,stores in tqdm(enumerate(geo_store['geometry'])):
    for name,area in zip(dong['EMD_KOR_NM'],dong['geometry']):
        if stores.within(area):
            geo_store.loc[idx,'동']=name

9915it [00:25, 382.24it/s]


In [90]:
# 비슷한 지역에 있을것이라 판단

geo_store.동.fillna(method='bfill',inplace=True)
geo_store.동.fillna(method='ffill',inplace=True)

# 병합
data=pd.merge(geo_store.reset_index(),store,on='gid',how='right')

In [145]:
dong['area']=dong.to_crs({'init':'epsg:4326'}).area

In [161]:
store_cnt=c_store.groupby('동',as_index=False).ws_cnt.sum()
store_cnt.columns=['EMD_KOR_NM','ws_cnt']

In [204]:
# 포화도(%) = 사업체 수 / 면적 *100 
print("포화도 Top 5")
display(c_store.동.value_counts(normalize=True))

포화도 Top 5


모현읍     0.106299
이동읍     0.062992
포곡읍     0.062992
남사면     0.049213
죽전동     0.043307
풍덕천동    0.039370
역북동     0.037402
신갈동     0.035433
유방동     0.033465
상현동     0.031496
보정동     0.029528
김량장동    0.027559
동천동     0.027559
영덕동     0.025591
양지면     0.025591
원삼면     0.025591
보라동     0.025591
신봉동     0.023622
중동      0.023622
성복동     0.021654
마북동     0.019685
마평동     0.019685
동백동     0.019685
구갈동     0.017717
고림동     0.017717
백암면     0.017717
상하동     0.015748
공세동     0.013780
언남동     0.013780
서천동     0.013780
청덕동     0.013780
삼가동     0.013780
고매동     0.011811
상갈동     0.009843
남동      0.005906
하갈동     0.003937
농서동     0.003937
고기동     0.003937
지곡동     0.003937
해곡동     0.001969
Name: 동, dtype: float64

In [169]:
data=gpd.read_file('용인3-6.geojson')

data['고령자비율']=data['고령자수']/data['총인구수']
data['생산가능인구비율']=data['생산가능인구']/data['총인구수']
data['유소년비율']=data['유소년수']/data['총인구수']

data=data.dropna().reset_index(drop=True)

dong=gpd.read_file('12.용인시_법정경계(읍면동).geojson')
dong.head()

# 구 분류
for idx,point in tqdm(enumerate(data['geometry'])):
    for name,area in zip(dong['EMD_KOR_NM'],dong['geometry']):
        if point.within(area):
            data.loc[idx,'EMD_KOR_NM']=name

# 뒤에걸로.
data['EMD_KOR_NM'].fillna(method='bfill',inplace=True)

7625it [00:17, 431.90it/s]


In [171]:
# 용인시 생산가능인구
display(data.groupby('EMD_KOR_NM',as_index=False)['생산가능인구'].sum())

,EMD_KOR_NM,생산가능인구
0,고기동,1836.0
1,고림동,16246.0
2,고매동,4482.0
3,공세동,12711.0
4,구갈동,30727.0
5,김량장동,13288.0
6,남동,2429.0
7,남사면,15877.0
8,농서동,2912.0
9,동백동,26102.0


In [172]:
pop=pd.read_csv('population.csv')

In [179]:
pop['EMD_KOR_NM']=pop['주소'].str.split(expand=True)[3]

In [181]:
time=['TMST_00', 'TMST_01', 'TMST_02', 'TMST_03', 'TMST_04','TMST_05',
      'TMST_06', 'TMST_07', 'TMST_08', 'TMST_09', 'TMST_10','TMST_11',
      'TMST_12', 'TMST_13', 'TMST_14', 'TMST_15', 'TMST_16','TMST_17',
      'TMST_18', 'TMST_19', 'TMST_20', 'TMST_21', 'TMST_22','TMST_23']

# 유동인구 어떻게 풀어야할까? 밴드로해서 0-9시 10-3시 3-7시 이런식?
pop.groupby(['EMD_KOR_NM'])[time].sum()

,TMST_00,TMST_01,TMST_02,TMST_03,TMST_04,TMST_05,TMST_06,TMST_07,TMST_08,TMST_09,...,TMST_14,TMST_15,TMST_16,TMST_17,TMST_18,TMST_19,TMST_20,TMST_21,TMST_22,TMST_23
EMD_KOR_NM,,,,,,,,,,,,,,,,,,,,,
고기동,1571.38,981.38,677.42,551.29,653.18,1787.58,5444.22,10487.24,11637.86,10806.72,...,11579.96,11375.92,11525.70,12739.72,12102.64,9242.66,6433.19,4939.71,3818.75,2467.51
고림동,7533.95,5606.91,4547.72,4202.87,5188.45,7672.94,13072.59,22566.48,28818.41,26162.61,...,28617.48,29621.83,31031.89,34562.87,34821.99,27092.58,22502.45,19073.52,14610.16,10135.70
고매동,8711.84,5709.87,4505.80,4270.39,5837.57,15117.35,33152.28,47134.27,47577.62,47177.21,...,57468.27,60648.96,65470.90,71037.89,61126.03,47335.38,36805.38,28630.82,21127.91,13089.46
공세동,6833.42,4525.75,3368.49,3002.38,3533.53,7363.88,17432.37,31486.62,33434.82,32673.77,...,37227.56,39087.94,41209.04,44136.08,38657.23,32283.13,26179.64,20429.10,15620.51,10452.52
구갈동,15438.78,10611.42,8393.73,7313.89,7356.07,11875.72,24018.50,39587.95,56345.30,48633.91,...,51352.68,54031.64,58672.97,68298.90,70191.94,61045.07,49627.16,43964.06,36267.90,24284.41
김량장동,8122.22,5862.23,4584.41,3935.92,4322.84,6726.86,12212.56,22754.65,32565.35,30197.04,...,33407.15,34214.84,35839.18,41564.40,42709.99,31810.17,25711.07,21654.78,16224.00,11358.68
남동,2355.54,1552.94,1133.72,1031.76,1613.77,3106.12,5965.54,11562.79,14164.92,12949.93,...,12332.36,12233.87,12870.70,13924.78,12710.70,9355.53,7397.94,6143.60,4622.79,3317.68
남사면,9440.60,6648.31,5328.99,5126.61,6647.95,13402.75,31562.72,56188.96,62106.89,53383.61,...,58071.50,58595.06,58414.29,60663.06,54206.25,40925.23,33075.01,26221.04,19708.86,13372.81
농서동,4124.83,3408.15,2638.00,2142.71,3795.57,12437.22,31471.36,48214.97,35199.39,23085.35,...,20933.13,20437.68,21604.38,26588.46,21065.52,15412.25,13471.87,13299.03,8815.96,5167.85


In [203]:
# 제조업과 정보통신업 농업 대출이력 비교. 이거 가능 파생변수같다.
# 정보통신
display(store[store.indcd_j_yn=='Y']['sme_loan_cnt'].sum())
# 제조업
display(store[store.indcd_c_yn=='Y']['sme_loan_cnt'].sum())
# 농업 
display(store[store.indcd_a_yn=='Y']['sme_loan_cnt'].sum())

2445

5426

98

In [189]:
loads=gpd.read_file('9.용인시_도로명주소_도로.geojson')

In [193]:
for idx,lines in tqdm(enumerate(loads['geometry'])):
    for name,area in zip(dong['EMD_KOR_NM'],dong['geometry']):
        if lines.within(area):
            loads.loc[idx,'EMD_KOR_NM']=name
loads['EMD_KOR_NM'].fillna(method='bfill',inplace=True)

5891it [00:15, 378.47it/s]


In [199]:
# 도로가 얼마나 많이 이어져있는지.
loads['EMD_KOR_NM'].value_counts()

# 다리 길이로 이어진거 곱하기하면 뭔가 조정될거같기도 하고.
# loads['EMD_KOR_NM']*0.6 + loads['geometry'].length * 0.4 이런식으로.
loads['geometry'].length

백암면     594
양지면     548
원삼면     525
모현읍     481
남사면     468
이동읍     454
포곡읍     370
김량장동    153
역북동     136
고기동     125
신갈동     119
유방동     109
풍덕천동    106
고림동     104
중동       95
구갈동      95
마평동      93
동천동      90
영덕동      76
보정동      75
신봉동      72
죽전동      70
남동       68
동백동      67
운학동      65
고매동      58
보라동      58
상하동      55
서천동      51
지곡동      49
마북동      48
청덕동      45
하갈동      44
언남동      43
삼가동      41
호동       40
상갈동      39
공세동      38
상현동      37
성복동      36
해곡동      26
농서동      25
Name: EMD_KOR_NM, dtype: int64

In [207]:
structures=gpd.read_file('용인8.geojson')

In [208]:
for idx,structure in tqdm(enumerate(structures['geometry'])):
    for name,area in zip(dong['EMD_KOR_NM'],dong['geometry']):
        if structure.within(area):
            structures.loc[idx,'EMD_KOR_NM']=name
structures['EMD_KOR_NM'].fillna(method='bfill',inplace=True)

100815it [06:37, 253.86it/s]


In [211]:
# 건물 수
structures['EMD_KOR_NM'].value_counts()

모현읍     10591
남사면      8908
양지면      8647
포곡읍      8033
이동읍      7813
백암면      7562
원삼면      7088
고림동      2173
김량장동     2099
신갈동      2026
역북동      1779
중동       1768
보정동      1734
고기동      1731
동천동      1682
고매동      1628
마평동      1598
풍덕천동     1568
유방동      1533
죽전동      1373
영덕동      1360
상하동      1297
남동       1251
지곡동      1174
마북동      1145
구갈동      1078
동백동      1070
보라동      1046
신봉동      1020
하갈동       901
공세동       896
운학동       865
삼가동       863
상현동       847
언남동       797
서천동       791
성복동       785
상갈동       615
청덕동       579
농서동       464
호동        446
해곡동       191
Name: EMD_KOR_NM, dtype: int64

In [4]:
realprice=pd.read_csv('realprice.csv',encoding='cp949',thousands=',')

In [5]:
realprice['EMD_KOR_NM']=realprice['소재지'].str.split(expand=True)[0]

In [12]:
fig,ax=plt.subplots(5,5,figsize=(10,6))
for idx,a in enumerate(alphabet):
    ax[idx//5][idx%5].bar(x=a.index,height=x)
    ax[idx//5][idx%5].plot(a)
    ax.set_xticks([])

,EMD_KOR_NM,이용상황,공시지가(원/m2)
503,신갈동,상업나지,5.700000e+06
722,죽전동,상업용,3.824364e+06
235,마북동,상업용,3.672222e+06
807,풍덕천동,상업용,3.554118e+06
504,신갈동,상업용,3.464000e+06
...,...,...,...
460,석천리,자연림,9.733333e+03
175,대대리,자연림,9.366667e+03
282,목신리,조림,7.100000e+03
825,학일리,조림,4.500000e+03


In [6]:
# 공시지가 
realprice.groupby(['EMD_KOR_NM','이용상황'],as_index=False)['공시지가(원/m2)'].mean()

,EMD_KOR_NM,이용상황,공시지가(원/m2)
0,가실리,골프장_대중제,88000.000000
1,가실리,유원지,416333.333333
2,가실리,자연림,20000.000000
3,가실리,전,99000.000000
4,가재월리,단독주택,213000.000000
...,...,...,...
838,화산리,공업용,304000.000000
839,화산리,단독주택,257000.000000
840,화산리,답,79000.000000
841,화산리,자연림,16275.000000


최적화 값을 곱할까?

In [124]:
# 제조업
c_store=data[data['indcd_c_yn']=='Y']

print("지역 별 제조업 사업체 총 합")
display(c_store.groupby('동').ws_cnt.sum())

cols=[col for col in c_store.columns if 'found_age' in col]
for i in range(6):
    col=cols[i]
    print(f"{i+1}년차 사업체 수 Top 5")
    display(c_store.groupby('동')[col].sum().sort_values(ascending=False)[:5])

print("휴/폐점 업 수 Top 5")
display(c_store.groupby('동')['runout_cnt'].sum().sort_values(ascending=False)[:5])

print("소상공인 사업장 수 Top 5")
display(c_store.groupby('동')['smbiz_yn_cnt'].sum().sort_values(ascending=False)[:5])

print("대표자가 남자인 사업장 수 Top 5")
display(c_store.groupby('동')['rpr_per_gender_m'].sum().sort_values(ascending=False)[:5])

print("대표자가 여자인 사업장 수 Top 5")
display(c_store.groupby('동')['rpr_per_gender_f'].sum().sort_values(ascending=False)[:5])

print("대표자가 10대인 사업장 수 Top 5")
display(c_store.groupby('동')['rpr_per_age_bin_10'].sum().sort_values(ascending=False)[:5])

print("대표자가 20대인 사업장 수 Top 5")
display(c_store.groupby('동')['rpr_per_age_bin_20'].sum().sort_values(ascending=False)[:5])

print("대표자가 30~50대인 사업장 수 Top 5")
display(c_store.groupby('동')['rpr_per_age_bin_30to50'].sum().sort_values(ascending=False)[:5])

print("대표자가 50대이상인 사업장 수 Top 5")
display(c_store.groupby('동')['rpr_per_age_bin_50over'].sum().sort_values(ascending=False)[:5])    

print("대출이력이 있는 사업장 수 Top 5")
display(c_store.groupby('동')['sme_loan_cnt'].sum().sort_values(ascending=False)[:5])  

print("2017년 3월 매출 사업장 순위 Top 5")
display(c_store.groupby('동')['sales_est_amt_201703'].sum().sort_values(ascending=False)[:5])    

print("이용자가 10대인 동 Top 5")
display(c_store.groupby('동')['age10_ratio'].sum().sort_values(ascending=False)[:5])

지역 별 제조업 사업체 수 Top 5


동
김량장동    1282
풍덕천동    1261
죽전동     1030
역북동      905
신갈동      884
Name: ws_cnt, dtype: int64

1년차 사업체 수 Top 5


동
역북동     51
동백동     36
상현동     33
풍덕천동    31
보정동     31
Name: found_age_1, dtype: int64

2년차 사업체 수 Top 5


...

동
풍덕천동    440
죽전동     425
김량장동    348
상현동     344
역북동     326
Name: sme_loan_cnt, dtype: int64

2017년 3월 매출 사업장 순위 Top 5


동
풍덕천동    628802
죽전동     383977
상현동     312107
김량장동    305406
보정동     283209
Name: sales_est_amt_201703, dtype: int64

이용자가 10대인 동 Top 5


동
역북동     33.72
풍덕천동    25.55
죽전동     23.35
신봉동     20.83
보라동     14.62
Name: age10_ratio, dtype: float64